In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

#define the features and target variable. don't include cabin, too many missing
y = train_data.Survived
features = ["Sex","Age","SibSp","Fare","Embarked"]
x = train_data[features]

#split into training and valid data
X_train,X_valid,y_train,y_valid = train_test_split(x,y,random_state = 0)

#define function for comparing results
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 100, random_state = 0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#177/860 people are missing age, i should try to do average age using fillna not imputer. also 2 empty values in embarked, replace wihh 0 
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_valid['Age'].fillna(X_valid['Age'].mean(), inplace=True)
X_train['Embarked'].fillna('Empty', inplace=True)
X_valid['Embarked'].fillna('Empty', inplace=True)

#apply OneHotEncoder to Sex and Embarked column becuase categorical variable
object_cols = ['Sex','Embarked']
OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

#OneHotEncoder removes index, put it back
OH_cols_train.index = X_train.index
OH_cols_train_valid = X_valid.index

#add encoded columns to dataset
OH_X_train = pd.concat([X_train, OH_cols_train], axis = 1)
OH_X_valid = pd.concat([X_valid, OH_cols_train], axis = 1)

#print MAE
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))



                                       
                                       




MAE from Approach 3 (One-Hot Encoding):


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


ValueError: could not convert string to float: 'male'